# Fast F1 - Save data

## Import libraries

In [ ]:
import fastf1
import matplotlib.pyplot as plt
import pandas as pd

from datetime import datetime

In [19]:
## TODO
## - Maybe reorganise the columns when saving or better leave for later preprocessing? 
##   But might be unneccessary disk space if I won't use the other columns.

# Specify the season year you want to process
year = 2024  # Replace with the desired season year

# Retrieve the event schedule for the specified season
schedule = fastf1.get_event_schedule(year)

# Filter sessions until current date
schedule = schedule[schedule['EventDate'] <= datetime.today()]

# Do not need the preseason results
schedule = schedule[schedule['EventName'] != 'Pre-Season Testing']

# Create an empty list to collect all laps data
all_laps = []

# Create an empty list to collect all race results data
all_race_results = []

# Create an empty list to collect all qualifying results data
all_quali_results = []

# Iterate over each event in the schedule
for _, event in schedule.iterrows():
    event_name = event['EventName']

    # Load the race session
    race_session = fastf1.get_session(year, event_name, 'R')
    race_session.load()  # Load the session data
    
    # Add the event name as a new column in the laps DataFrame
    laps = race_session.laps
    laps['EventName'] = event_name
    
    # Append the modified laps data to the list
    all_laps.append(laps)
    print(f"Laps data loaded for {event_name}")

    # Add the event name as a new column in the results DataFrame
    race_results = race_session.results
    race_results['EventName'] = event_name

    # Append the modified race results data to the list
    all_race_results.append(race_results)
    print(f"Race results data loaded for {event_name}")

    # Load the qualifying session
    quali_session = fastf1.get_session(year, event_name, 'Q')
    quali_session.load()
    
    # Add qualifying results data
    quali_results = quali_session.results
    quali_results['EventName'] = event_name

    # Append the modified qualifying results data to the list
    all_quali_results.append(quali_results)
    print(f"Qualifying results data loaded for {event_name}")

# Concatenate all collected laps data into a single DataFrame
if all_laps:
    all_laps_df = pd.concat(all_laps, ignore_index=True)
    
    # Save the DataFrame to a CSV file
    output_file = f'data/gps_{year}_season_laps.csv'
    all_laps_df.to_csv(output_file, index=False)
    print(f"Laps data for {year} saved to {output_file}")
else:
    print("No laps data available for the selected season.")

# Save race results data to a CSV file
if all_race_results:
    all_race_results_df = pd.concat(all_race_results, ignore_index=True)
    race_results_output_file = f'data/gps_{year}_season_results.csv'
    all_race_results_df.to_csv(race_results_output_file, index=False)
    print(f"Race results data for {year} saved to {race_results_output_file}")
else:
    print("No race results data available for the selected season.")

# Save qualifying results data to a CSV file
if all_quali_results:
    all_quali_results_df = pd.concat(all_quali_results, ignore_index=True)
    quali_results_output_file = f'data/gps_{year}_season_quali.csv'
    all_quali_results_df.to_csv(quali_results_output_file, index=False)
    print(f"Qualifying results data for {year} saved to {quali_results_output_file}")
else:
    print("No qualifying results data available for the selected season.")


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.4.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Bahrain Grand Prix - Qua

Laps data loaded for Bahrain Grand Prix
Race results data loaded for Bahrain Grand Prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2', '31', '10']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.4.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Qualifying results data loaded for Bahrain Grand Prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '81', '14', '63', '38', '4', '44', '27', '23', '20', '31', '2', '22', '3', '77', '24', '18', '10']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Qualifying [v3.4.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


...

Qualifying results data loaded for São Paulo Grand Prix


core        WARNING 	Driver 63: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver 55: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 16: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver  1: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  4: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 81: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 44: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 30: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver 77: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Cannot load lap times for first lap from Ergast. Timing data is not available for this session.
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req    

Laps data loaded for Las Vegas Grand Prix
Race results data loaded for Las Vegas Grand Prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '55', '10', '16', '1', '4', '22', '81', '27', '44', '31', '20', '24', '43', '30', '11', '14', '23', '77', '18']


Qualifying results data loaded for Las Vegas Grand Prix
Laps data for 2024 saved to data/gps_2024_season_laps.csv
Race results data for 2024 saved to data/gps_2024_season_results.csv
Qualifying results data for 2024 saved to data/gps_2024_season_quali.csv
